### Python script to merge GPS information form AWI Polar-X planes
author Ingmar Nitze - ingmar.nitze@awi.de

#### ToDo List
1. points to line conversion **Done**
2. only load LAT/LON (simplify loading)
3. Create wrapping function

In [37]:
import pandas as pd
import os
import glob
import geopandas as gpd
import fiona
from shapely.geometry import Point, LineString, shape
fiona.drvsupport.supported_drivers['KML'] = 'rw' 

In [38]:
def load_file(f):
    cname = (os.path.basename(f).split('.')[1]).split('__')[0]
    df = pd.read_table(f, delim_whitespace=True, header=None, names=['timestamp', cname], index_col='timestamp')
    return df

In [39]:
def to_decimal_degrees(number, factor=1):
    s = str(number)
    splitlen = len(s.split('.')[0])
    float(s[splitlen-2:])/60
    return float(s[:splitlen-2]) + float(s[splitlen-2:])/60

In [40]:
#DIRECTORY = r'C:\Users\initze\OneDrive\136_Expedition_Planning\2021_PermaX_WestAlaska\GPS_Export\GPS1'
DIRECTORY = r'N:\Response\Restricted_Airborne\MACs\2021_Perma-X_Alaska\00_GPS\GPS_1'

In [41]:
glob.glob(DIRECTORY + '/*')

['N:\\Response\\Restricted_Airborne\\MACs\\2021_Perma-X_Alaska\\00_GPS\\GPS_1\\2107010701',
 'N:\\Response\\Restricted_Airborne\\MACs\\2021_Perma-X_Alaska\\00_GPS\\GPS_1\\2107020801',
 'N:\\Response\\Restricted_Airborne\\MACs\\2021_Perma-X_Alaska\\00_GPS\\GPS_1\\2107030901',
 'N:\\Response\\Restricted_Airborne\\MACs\\2021_Perma-X_Alaska\\00_GPS\\GPS_1\\2107091001',
 'N:\\Response\\Restricted_Airborne\\MACs\\2021_Perma-X_Alaska\\00_GPS\\GPS_1\\2107101101']

In [42]:
directory = glob.glob(DIRECTORY + '/*')[-2]

In [43]:
filename = os.path.split(directory)[-1]

#### Create filelist and import to Dataframe

In [44]:
flist = glob.glob(directory + '/*/*.dat')

In [45]:
df_list = []
for f in flist:
    try:
        df_tmp = load_file(f)
        df_list.append(df_tmp)
    except:
        continue

In [46]:
df = pd.concat(df_list, axis=1)

#### Recalculate and convert value to correct format 

In [47]:
df_ll = df.dropna(subset=['LON']).drop_duplicates().reset_index().dropna(axis=1)
df_ll['LON'] = df_ll['LON'].apply(to_decimal_degrees)*-1
df_ll['LAT'] = df_ll['LAT'].apply(to_decimal_degrees)

In [48]:
geom = gpd.points_from_xy(df_ll.LON, df_ll.LAT)

#### Create Geodataframe and export


In [49]:
gdf = gpd.GeoDataFrame(df_ll, geometry=geom)

In [50]:
outfile = f'{filename}.geojson'
gdf.to_file(outfile, driver='GeoJSON')

#### Convert points to track/Line 

In [51]:
outfile_line_json = f'{filename}_line.geojson'
outfile_line_kml = f'{filename}_line.kml'

In [52]:
gdf_line = LineString(gdf.geometry.tolist())
gds = gpd.GeoSeries(data=gdf_line)

In [53]:
if not os.path.exists(outfile_line_json):
    gds.to_file(outfile_line_json, driver='GeoJSON')
else:
    print('File already exists')

In [54]:
if not os.path.exists(outfile_line_kml):
    gds.to_file(outfile_line_kml, driver='KML')
else:
    print('File already exists')